In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/lib/spark/"
os.environ["PYSPARK_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["SPARK_YARN_USER_ENV"]= "/home/cloudera/anaconda2/bin/python"

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

from pyspark import SparkContext, SparkConf

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:09:15)
SparkContext available as sc, HiveContext available as sqlContext.


##### The Kaggle Instatcart Market Basket analysis has data with more than 2 million rows in orders_product_prior and other datat set also has around a milloin rows that requires manipulation using partition and parallel processing.
##### For this reason In am using spark in jupyter and building my complete project in pycharm IDE. 
##### Created a virtual enviroment with spark libraries and anaconda python 2.7, initial settings are shown inthe first cell. 
##### Changed the 'dfs.blocksize' paramemter in the hdfs-default.xml to 64X1024X1024 bytes. 
##### This will increase parallisum in the hdfs processing for aggregating dataset in memory becomes faster.
##### for more information check this blog http://www.bigsynapse.com/spark-input-output.
##### Though I am using Pseudorandom distribution on cloudera quick start VM 5.8 it might not have any performance benifit as expected as machine is single but one cn try increasing the no of core and the executor memory.
##### default driver memory given is 5g in my VM , since in local mode executor is inside the driver so can't get memory more than that.##### underline properties to configure is spark  --num-executors <is one as installation is standalone> --executor-cores <depends on core of processor> --executor-memory <Memory available to the VM>

#### Order Schema 
##### order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_orde
##### read data with partition of three since data size of orders file is more than 110 mb we use 4 partition of size 3*32 mb
##### 32 mb is the local file system block size in whicj data is stored
##### 128 mb block size is the size of memory the name node referrences more 32 size blockes can be added to it.
##### It is for the namenode to decide in which 128 mb block refrence the 32 mb block can go

In [6]:
orders = sc.textFile("file:///home/cloudera/Desktop/kaggle/orders.csv", 4).filter(lambda x: 'order_id' not in x).map(lambda x : x.split(','))

In [15]:
orders.take(1)

[[u'2539329', u'1', u'prior', u'1', u'2', u'08', u'']]

#### Order_Product_Schema
##### order_id, Product_id, 
##### Here for we are taking the partition of 'order_products__prior' to be 18 as it is of more 550 mb 
##### similarly for 'order_products__train'  it is  default 0 partiton as the size is less than block size 32 mb

In [10]:
orders_prior = sc.textFile("file:///home/cloudera/Desktop/kaggle/order_products__prior.csv", 18).filter(lambda x: 'order_id' not in x).\
    map(lambda x : x.split(','))

orders_train = sc.textFile("file:///home/cloudera/Desktop/kaggle/order_products__train.csv").filter(lambda x: 'order_id' not in x).\
    map(lambda x : x.split(','))

In [21]:
orders_prior.take(1)

[[u'2', u'33120', u'1', u'1']]

##### Save all rdds created to hadoop file system in distributed fashion based on the no of the partions inthe rdd while loading 

In [23]:
orders.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders")

In [24]:
orders_prior.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders_prior")

In [25]:
orders_train.map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/orders_train")

In [28]:
## Define null prser to parse the blank strings 
def nullparse(x):
    if x =='':
        x=0
    else:
        float(x)
    return x

In [41]:
x= sc.textFile("/user/cloudera/kaggle/orders").map(lambda x: x.split(',')).map(lambda x : (int(x[0]),(x[1],x[2],x[3],x[4],x[5],str(nullparse(x[6])))))

In [38]:
y=sc.textFile("/user/cloudera/kaggle/orders_prior").map(lambda x: x.split(',')).map(lambda x : (int(x[0]), x[1]))

In [ ]:
z= sc.textFile("/user/cloudera/kaggle/orders_train").map(lambda x: x.split(',')).map(lambda x : (int(x[0]), x[1]))

In [43]:
x.filter(lambda x : x[1][1]!="test").mapValues(lambda x : x[5]).reduceByKey(lambda x,y : x+y).\
    sortBy(lambda x : -x[1]).take(3)

#### Combine all products purchased for a perticular order id in one string sperated by space for order_product_prior data set.

In [45]:
y.groupByKey().mapValues(lambda x : ' '.join(x)).take(3)

[(3145734, u'45223 32740 46616 8230 6975 47668 3582'),
 (2621448, u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887'),
 (2097162, u'48988 37774 24852 19057')]

#### Save the intermediate result to the HDFS to reduce the time for further agregations

In [52]:
y.groupByKey().mapValues(lambda x : ' '.join(x)).map(lambda x : (str(x[0]),x[1])).map(lambda x : ','.join(x)).saveAsTextFile("/user/cloudera/kaggle/order_prior_inter")

In [53]:
order_prod_prio_inter = sc.textFile("/user/cloudera/kaggle/order_prior_inter").map(lambda x: x.split(','))

In [54]:
order_prod_prio_inter.take(2)

[[u'3145734', u'45223 32740 46616 8230 6975 47668 3582'],
 [u'2621448', u'37718 38544 2748 47209 45504 28699 8006 41665 2078 35887']]

In [8]:
order_prod_prio_inter

In [56]:
x.take(2)

[(2539329, (u'1', u'prior', u'1', u'2', u'08', '0')),
 (2398795, (u'1', u'prior', u'2', u'3', u'07', '15.0'))]